# To-Do List

- Abstract code into functions
- DONE Script to create one big dataframe containing all data from all races currently raced this season
- DONE Fetch qualificiation data
- Train model
- Live training and prediction
- Dashboard

# Imports

In [ ]:
import fastf1
from fastf1 import Cache
import pandas as pd
import numpy as np
import streamlit as st
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import os

# Data Fetching

In [19]:
current_year = 2025

schedule = fastf1.get_event_schedule(current_year)

data = []
labels = []
for _, row in schedule.iterrows():
    if row['EventFormat'] != 'conventional': 
        continue
    session = fastf1.get_session(current_year, row['EventName'], 'R')
    session.load()
    session.load(telemetry=False, weather=False)
    laps = session.laps.pick_accurate().reset_index(drop=True)
    if laps.empty:
        continue
    for driver in session.drivers:
        driver_laps = laps[laps['Driver'] == driver]
        total_time = driver_laps['LapTime'].sum().total_seconds()
        grid_position = driver_laps['GridPosition'].iloc[0] if not drvlaps.empty else np.nan
        final_position = driver_laps['Position'].iloc[-1] if not driver_laps.empty else np.nan
        team = driver_laps['Team'].iloc[0]
        data.append([driver, team, grid_position, final_position, total_time])
df = pd.DataFrame(data, columns=['Driver', 'Team', 'GridPosition', 'FinalPosition', 'TotalRaceTime'])
df['Label'] = labels
os.makedirs("data", exist_ok=True)
df.to_csv(f"data/historical_data_{years[0]}.csv", index=False)

df.head()

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
req            INFO 	Data has been written to cache!
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req           

IndexError: single positional indexer is out-of-bounds

# Data Preprocessing

In [ ]:
df = df.copy()
le_driver = LabelEncoder()
le_team = LabelEncoder()
df['DriverEncoded'] = le_driver.fit_transform(df['Driver'])
df['TeamEncoded'] = le_driver.fit_transform(df['Team'])
df = df[['DriverEncoded', 'TeamEncoded', 'GridPosition', 'FinalPosition', 'TotalRaceTime']]

# Static Model Training

# Live Model Updating

# Model Dashboard